In [1073]:
# creating the classes

class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = {}

    def add_room(self, room):
        self.rooms[room.name] = room

    def add_door(self, door):
        if door.room1.name in self.rooms and door.room2.name in self.rooms:
            self.doors[door.name] = door
            door.room1.add_door(door)
            door.room2.add_door(door)
        

class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []

    def add_door(self, door):
         if door not in self.doors:
             self.doors.append(door)
        # self.doors[door.name] = door

    def add_object(self, obj):
        self.objects[obj.name] = obj

    def search(self):
        if self.objects:
            print("You found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("There's nothing interesting here.")
        if self.doors:
             print("There are also:")
             for door in self.doors:
                other_room = door.get_other_room(self)
                print(f"- {door.name}")  
        else:
             print("There are no doors here.")

class Object:
    def __init__(self, name, description, item=None):
        self.name = name
        self.description = description
        self.item = item

    def interact(self, player):
        if self.item:
            print(f"You found a {self.item.name} inside the {self.name}!")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Chest(Object):
    def __init__(self, name, description, item = None, is_locked=True, correct_pin = None):
        super().__init__(name, description, item)
        self.is_locked = is_locked
        self.correct_pin = correct_pin 
    
    def interact(self, player):
        if self.is_locked:
            print(f"The {self.name} is locked, try to open it.! Pin-code is {self.correct_pin}")
            while True:
                pin = input()
                if pin.lower() == "cancel":
                    print("You decided not to try the PIN code.")
                    return  
                try:
                    if len(pin) != 4 or not pin.isdigit():  # check length and if cod contains digits
                        raise ValueError
                    if int(pin) == self.correct_pin:  # check if the pin is correct
                        self.is_locked = False
                        print(f"You unlocked the {self.name}!")
                        super().interact(player)  # open the chest 
                        return  
                    else:
                        print("Incorrect PIN. Try again or type 'cancel':")
                except ValueError:
                    print("Invalid input. Please enter a 4-digit PIN or type 'cancel':")

        else: 
            print("It looks like someone already opened it. There's nothing interesting here.")
    

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, name, room1, room2, key_required = None):
        self.name = name
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []


    def move(self, door):
        if door.key_required and door.key_required not in self.inventory:
            print(f"You need a {door.key_required.name} to go there.")
            return
        else:
            new_room = door.get_other_room(self.current_room)
            self.current_room = new_room
            print(f"you moved to {self.current_room.name}. {self.current_room.description}")
            return
                 

    
    def search_room(self):
        self.current_room.search()

    def interact_with(self, object_name):
        if object_name in self.current_room.objects:
            obj = self.current_room.objects[object_name]
            obj.interact(self)
        else:
            print(f"There's no {object_name} here.")

In [1074]:
# creating & setting the game

# Create a house
house = HauntedMansion()

# Create rooms
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.")
hallway = Room("Hallway", "An empty hallway.")
bedroom = Room("Bedroom", "A peaceful bedroom with a soft bed.")
dining_room = Room("Dining Room", "A nice room to have a meal.")
outside = Room("Outside", "A breeze of fresh air and singing birds. Congratulations, you've made it to the outside and won the game!")

silver_key = Item("Silver key", "Opens the door between the dining room and the kitchen")
golden_key = Item("Golden key", "Opens the door between the hallway and the dining room")
master_key = Item("Master key", "Opens the big door")


# Add rooms to the house
house.add_room(kitchen)
house.add_room(hallway)
house.add_room(bedroom)
house.add_room(dining_room)
house.add_room(outside)


silver_door = Door("Silver door", dining_room, kitchen, silver_key)
white_door = Door("White door", dining_room, hallway)
gold_door = Door("Gold door", hallway, bedroom, golden_key)
big_door = Door("Big door", hallway, outside, master_key)



# Connect rooms with doors
house.add_door(silver_door)
house.add_door(white_door)
house.add_door(gold_door)
house.add_door(big_door)



# Create a player and set their starting room
player = Player()
player.current_room = dining_room

# Create a drawer that contains a key
chest = Chest("Chest", "A mysterious chest.", golden_key, correct_pin = 1920)
drawer = Object("Drawer", "An old wooden drawer.", silver_key)
bed = Object("Bed", "A large bed.", master_key)

# Add the drawer to the kitchen
dining_room.add_object(drawer)
kitchen.add_object(chest)
bedroom.add_object(bed)

In [1075]:
print(f"You are in the {player.current_room.name}.")

You are in the Dining Room.


In [1076]:
player.search_room()

You found:
- Drawer: An old wooden drawer.
There are also:
- Silver door
- White door


In [1077]:
player.move(silver_door)

You need a Silver key to go there.


In [1078]:
player.move(white_door)

you moved to Hallway. An empty hallway.


In [1079]:
player.search_room()

There's nothing interesting here.
There are also:
- White door
- Gold door
- Big door


In [1080]:
player.move(white_door)

you moved to Dining Room. A nice room to have a meal.


In [1081]:
player.search_room()

You found:
- Drawer: An old wooden drawer.
There are also:
- Silver door
- White door


In [1082]:
player.interact_with("Drawer")

You found a Silver key inside the Drawer!


In [1083]:
player.move(silver_door)

you moved to Kitchen. A warm kitchen with a smell of fresh bread.


In [1084]:
player.search_room()

You found:
- Chest: A mysterious chest.
There are also:
- Silver door


In [1085]:
player.interact_with("Chest")

The Chest is locked, try to open it.! Pin-code is 1920
You decided not to try the PIN code.


In [1086]:
player.move(silver_door)

you moved to Dining Room. A nice room to have a meal.


In [1087]:
player.move(white_door)

you moved to Hallway. An empty hallway.


In [1088]:
player.search_room()

There's nothing interesting here.
There are also:
- White door
- Gold door
- Big door


In [1089]:
player.move(gold_door)

You need a Golden key to go there.


In [1090]:
player.search_room()

There's nothing interesting here.
There are also:
- White door
- Gold door
- Big door


In [1091]:
player.interact_with("Bed")

There's no Bed here.


In [1092]:
player.move(gold_door)

You need a Golden key to go there.


In [1093]:
player.move(big_door)

You need a Master key to go there.
